# Agent Development Notebook

In [1]:
%%bash
cd frontend && npm install && npm run dev


up to date, audited 1081 packages in 1s

348 packages are looking for funding
  run `npm fund` for details

14 moderate severity vulnerabilities

To address issues that do not require attention, run:
  npm audit fix

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.

> frontend@0.1.0 dev
> next dev

▲ Next.js 16.1.6 (Turbopack)
- Local:         http://localhost:3000
- Network:       http://10.100.1.217:3000
- Environments: .env

✓ Starting...
✓ Ready in 520ms
○ Compiling / ...
 GET / 200 in 8.5s (compile: 8.1s, render: 443ms)
 POST /api/copilotkit 404 in 1498ms (compile: 1477ms, render: 21ms)
 POST /api/copilotkit 404 in 1499ms (compile: 1479ms, render: 20ms)
 POST /api/copilotkit 404 in 1527ms (compile: 1507ms, render: 20ms)
 POST /api/copilotkit 404 in 1526ms (compile: 1506ms, render: 20ms)
 POST /api/copilotkit 404 in 1444ms (compile: 1423ms, render: 20ms)
 POST /api/copilotkit 404 in 1497ms (compile: 1476ms, render: 20ms

In [ ]:
import threading
import uvicorn
from ag_ui_langgraph import add_langgraph_fastapi_endpoint
from copilotkit import LangGraphAGUIAgent
from fastapi import FastAPI
from langgraph.checkpoint.memory import MemorySaver

app = FastAPI()
memory = MemorySaver()

agent = LangGraphAGUIAgent(
    name="sample_agent",
    description="An example agent.",
    graph=None,
)
add_langgraph_fastapi_endpoint(app=app, agent=agent, path="/")

threading.Thread(
    target=lambda: uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning"),
    daemon=True,
).start()

print("Server running at http://localhost:8000")

In [ ]:
from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI


async def chat(state: MessagesState):
    model = ChatOpenAI(model="gpt-4o-mini")
    system_message = SystemMessage(content="You are a helpful assistant.")
    response = await model.ainvoke([system_message, *state["messages"]])
    return {"messages": response}


memory = MemorySaver()
graph = StateGraph(MessagesState)
graph.add_node(chat)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

agent.graph = graph.compile(checkpointer=memory)
print("Agent updated!")